# Exercise 2 - QOSF Mentorship program

<hr>

**Task:**
* Prepare 4 random 4-qubit quantum states of your choice.
- Create and train a variational circuit that transforms input states into predefined output states. Namely
  - if random state 1 is provided, it returns state |0011>
  - if random state 2 is provided, it returns state |0101>
  - if random state 3 is provided, it returns state |1010>
  - if random state 4 is provided, it returns state |1100>
- What would happen if you provided a different state?

Analyze and discuss the results.

<hr>

## Solution

In [1]:
# import modules
import pennylane as qml
import numpy as np
import torch
from torch.autograd import Variable
from torch.nn.functional import normalize

In [2]:
# circuit characteristics of the circuit
nr_qubits = 4
nr_layers = 1
nr_params = 6

In [3]:
# function to create the normalized random input states
def create_input_state(qubits_states):

    if len(qubits_states) != nr_qubits:
        print('Error: expected {} qubits states, {} where provided.'.format(nr_qubits, len(qubits_states)))
        return
    
    normalized_states = [normalize(q, p=2, dim=0) for q in qubits_states]

    input_state = torch.kron(normalized_states[0], normalized_states[1])
    
    for i in range(2, nr_qubits):
        input_state = torch.kron(input_state, normalized_states[i])
    
    return input_state

In [4]:
# input and target states definition

zero = torch.tensor([1., 0.])
one  = torch.tensor([0., 1.])


# random input states (in computational basis)
input_states = {}

for i in range(4):
    input_states[i] = create_input_state([torch.rand(2) + torch.rand(2)*1.j for _ in range(nr_qubits)])
'''
input_states[0] = create_input_state([one, zero, zero, zero])
input_states[1] = create_input_state([zero, one, zero, zero])
input_states[2] = create_input_state([zero, zero, one, zero])
input_states[3] = create_input_state([zero, zero, zero, zero])
'''

# target output states (on the bloch sphere)
zero = [0., 0.,  1.]
one  = [0., 0., -1.]

target_states = {}

target_states[0] = torch.tensor([zero, zero, one, one])
target_states[1] = torch.tensor([zero, one, zero, one])
target_states[2] = torch.tensor([one, zero, one, zero])
target_states[3] = torch.tensor([one, one, zero, zero])

nr_states = len(target_states)

# normalization of each target state
for target_state in target_states.values():
    for i in range(len(target_state)):
        target_state[i] = normalize(target_state[i], p=2, dim=0)
        
# wrap input and target states in a useful structure
states = {}
for i in range(len(target_states)):
    states[i] = {'input': input_states[i],
                 'target': target_states[i]}

In [5]:
# array of Pauli matrices
Paulis = Variable(torch.zeros([3, 2, 2], dtype=torch.complex128), requires_grad=False)
Paulis[0] = torch.tensor([[0, 1], [1, 0]])
Paulis[1] = torch.tensor([[0, -1j], [1j, 0]])
Paulis[2] = torch.tensor([[1, 0], [0, -1]])

In [6]:
# initialize parameters to random values
params = np.random.uniform(0, np.pi, (nr_qubits, nr_layers, nr_params))
params = Variable(torch.tensor(params), requires_grad=True)

In [7]:
# definition of different layers to be applied in the circuit
def rotation_layer(params, j):
    for i in range(nr_qubits):
        qml.RX(params[i, j, 0], wires=i)
        qml.RY(params[i, j, 1], wires=i)

def crot_layer(params, j):    
    for i in range(nr_qubits):
        for k in range(i+1, nr_qubits):
            qml.CRX(params[i, j, 2], wires=[i, k])
            qml.CRX(params[i, j, 3], wires=[k, i])
            qml.CRY(params[i, j, 4], wires=[k, i])
            qml.CRY(params[i, j, 5], wires=[i, k])
                
def cnot_layer():
    for i in range(nr_qubits - 1):
        qml.CNOT(wires=[i, i+1])
        

In [8]:
# select device
dev = qml.device("default.qubit", wires=nr_qubits)

In [9]:
# circuit definition
@qml.qnode(dev, interface="torch")
def circuit(params, A, wire, input_state):

    qml.QubitStateVector(input_state, wires=range(nr_qubits))
    
    
    # apply layers in the circuit
    for j in range(nr_layers):
        rotation_layer(params, j)
        cnot_layer()
        crot_layer(params, j)
            
    # returns the expectation of the input matrix A on the given wire
    return qml.expval(qml.Hermitian(A, wires=wire))

In [10]:
# cost function for single computed and target couple states
def cost_fn_single_state(params, state):
    cost = 0
    for wire in range(nr_qubits):
        for k in range(3):
            cost += torch.abs(circuit(params, Paulis[k], wire, state['input']) - state['target'][wire][k])            

    return cost / nr_qubits

In [11]:
# cost function
# cost is the sum of the difference between each pair of computed and target states 
def cost_fn(params):
    cost = 0
    for state in states.values():
        cost += cost_fn_single_state(params, state)

    return cost / nr_states

The following cells contain the code for the optimization. They are meant to executed multiple times, manually lowering the learning rate: this example has been trained starting from $lr = 0.1$ and finishing at $lr = 0.001$.

In [12]:
total_steps = 0

# the final stage of optimization isn't always the best, so we keep track of
# the best parameters along the way
best_cost = cost_fn(params)
best_params = params

In [ ]:
# Optimization
# set up the optimizer
opt = torch.optim.Adam([params], lr=0.1)

# number of steps in the optimization routine
steps = 100

# optimization begins
for n in range(steps):
    opt.zero_grad()
    loss = cost_fn(params)
    loss.backward()
    opt.step()
    
    # keeps track of best parameters
    if loss < best_cost:
        best_cost = loss
        best_params = params

    # Keep track of progress every 10 steps
    #if n % 10 == 9 or n == steps - 1:
    print("Best cost after {} steps is {:.4f}".format(total_steps + n + 1, best_cost))
    
params = best_params
total_steps += steps

Here it follows a comparison between the output states from the trained circuit and the target states. Results are not always correct, possibly due to the circuit structure and the small number of parameters.

In [14]:
# calculate the Bloch vector of the output states and confront with target states
output_state = [torch.zeros((nr_qubits, 3)) for _ in range(nr_states)]
for key, state in states.items():
    for wire in range(nr_qubits):
        for l in range(3):
            output_state[key][wire][l] = circuit(best_params, Paulis[l], wire, state['input'])

    for i in range(len(output_state[key])):
        output_state[key][i] = normalize(output_state[key][i], p=2, dim=0)

    # print results
    print('Results for input/target states number {}'.format(key+1))
    print("Target Bloch vector: \n", target_states[key])
    print("Output Bloch vector: \n", output_state[key])
    print("Cost = ", float(cost_fn_single_state(best_params, states[key])))
    print('\n')

Results for input/target states number 1
Target Bloch vector: 
 tensor([[ 0.,  0.,  1.],
        [ 0.,  0.,  1.],
        [ 0.,  0., -1.],
        [ 0.,  0., -1.]])
Output Bloch vector: 
 tensor([[-0.2281,  0.2725,  0.9347],
        [-0.2641,  0.0514,  0.9631],
        [ 0.0436, -0.1491, -0.9879],
        [ 0.0118,  0.0277, -0.9995]], grad_fn=<CopySlices>)
Cost =  0.713364890870345


Results for input/target states number 2
Target Bloch vector: 
 tensor([[ 0.,  0.,  1.],
        [ 0.,  0., -1.],
        [ 0.,  0.,  1.],
        [ 0.,  0., -1.]])
Output Bloch vector: 
 tensor([[-0.0846,  0.7342,  0.6736],
        [ 0.0533,  0.6494, -0.7586],
        [ 0.9183,  0.0908, -0.3854],
        [ 0.0580, -0.6551, -0.7534]], grad_fn=<CopySlices>)
Cost =  1.0786525638434856


Results for input/target states number 3
Target Bloch vector: 
 tensor([[ 0.,  0., -1.],
        [ 0.,  0.,  1.],
        [ 0.,  0., -1.],
        [ 0.,  0.,  1.]])
Output Bloch vector: 
 tensor([[-0.1615, -0.4640, -0.8710],


In [15]:
print(best_params)

tensor([[[ 1.7134, -0.0705,  1.5856,  0.9543,  1.1344,  2.7869]],

        [[ 0.9208,  1.2785,  2.4886,  3.0529,  1.1519,  0.6474]],

        [[ 0.9173,  2.0862,  2.7326,  2.0511,  1.8842,  1.0658]],

        [[ 0.9805,  1.3092,  0.7695,  0.0805,  0.4428,  2.0019]]],
       dtype=torch.float64, requires_grad=True)


<hr>

**What would happen if you provided a different state?**

The target states given by the task are linearly independent and constitutes a basis for a subspace of dimention 4 in the computational basis, which has dimention $2^{nr\_qubits} = 16$. The same holds for the random input states, except that it's not guaranteed that they all are linearly independent, and we can extract a basis of at most 4 vectors.

The unitary map implemented by the circuit is then defined between theese two basis, but random elsewhere. If a new random state is provided, its components on the input basis are mapped to respective states in the target basis, with the same linear combination, while other components are mapped to fixed but random states.